In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow # Colab 환경에서 이미지 표시를 위한 함수 임포트
import time # 출력 속도 조절을 위해 임포트

# --- 1. 플로 시각화 함수 정의 ---
def drawFlow(img, flow, step=16):
    """
    Farneback 옵티컬 플로우 결과를 시각화하는 함수.
    특정 간격(step)마다 화살표를 그려 움직임 벡터를 표시합니다.
    """
    h, w = img.shape[:2]
    # .shape >> (높이, 너비, 채널)
    # .shape[:2] 높이, 너비

    # 16픽셀 간격의 그리드 인덱스 구하기 (좌표: y, x)

    idx_y, idx_x = np.mgrid[step//2:h:step, step//2:w:step].astype(np.int32)
    # step//2: 시작 지점을 정수 나눗셈으로 안전하게 지정
    # 시작점: step//2 8, 끝 h (이미지 높이), 간격 (step) 16
    # 결과 [ 8, 24, 40, 56, ... ]

    # y, x를 묶어 (N, 2) 형태의 좌표 목록 (x, y 순서)으로 재구성
    indices = np.stack((idx_x, idx_y), axis=-1).reshape(-1, 2)

    # 화살표 색상 (녹색)
    arrow_color = (0, 255, 0)

    for x, y in indices:    # 각 그리드 인덱스 순회 (x: 열, y: 행)
        # 1. 각 그리드 인덱스 위치에 시작점(점) 그리기 (녹색)
        cv2.circle(img, (x, y), 1, arrow_color, -1)

        # 2. 각 그리드 인덱스에 해당하는 플로 결과 값(이동 거리) 얻기
        # flow 배열은 (H, W, 2) 형태이며, (dx, dy) 벡터를 담고 있습니다.
        # dx, dy는 float 형태이므로 정수형으로 변환
        dx, dy = flow[y, x].astype(np.int32)
        # flow[y, x] 해당 위치의 움직임 array(배열) [행, 열] = [y, x]
        # dx : x축 방향의 이동거리 (오른쪽 +, 왼쪽 -)
        # dy : y축 방향의 이동거리 (아래 +, 위 -)

        # 3. 각 그리드 인덱스 위치에서 이동한 거리만큼 선(화살표) 그리기
        # 시작점: (x, y), 끝점: (x+dx, y+dy)
        end_point = (x + dx, y + dy)

        # cv2.line 대신 cv2.arrowedLine() 함수를 사용하여 움직임 벡터를 명확하게 시각화합니다.
        cv2.arrowedLine(img, (x, y), end_point, arrow_color, 1, cv2.LINE_AA, tipLength=0.3)
        # cv2.LINE_AA : anti-aliasing 부드러운 선
        # tipLength=0.3 화살촉 크기 (전체의 30%)


prev = None # 이전 프레임 저장 변수 (그레이스케일)

In [3]:
#  연습
# step_ex = 16
# step_ex // 2 # 8 (16의 절반)

x_idx = [10,20,30]
y_idx = [20,40,60]

# np.stack((x_idx, y_idx), axis=0)
'''
array([[10, 20, 30],
       [20, 40, 60]])

'''

# np.stack((x_idx, y_idx), axis=1)
'''
array([[10, 20],
       [20, 40],
       [30, 60]])
'''
# np.stack((x_idx, y_idx), axis=-1)
'''
array([[10, 20],
       [20, 40],
       [30, 60]])
'''

# np.stack((x_idx, y_idx), axis=-1).shape  # (3,2)

# np.stack((x_idx, y_idx), axis=-1).reshape(-1,2) # 동일한 결과(동일한 형태)

'\narray([[10, 20],\n       [20, 40],\n       [30, 60]])\n'

In [4]:
# --- 2. 비디오 캡처 설정 ---
# NOTE: Colab에서 이 코드를 실행하기 전에, 'newyork.mp4' 파일을 Colab 환경에 업로드해야 합니다.
video_path = '/content/drive/MyDrive/두산로보틱스_딥러닝_컴퓨터비전/7기_컴퓨터비전 응용/교육생제공용/data/newyork.mp4'
cap = cv2.VideoCapture(video_path) # 비디오 캡처 객체 생성
delay = int(1000/30) # 프레임 재생 속도 조절을 위한 딜레이

# 비디오 파일이 제대로 열렸는지 확인
if not cap.isOpened():
    print("오류: 비디오 파일을 열 수 없습니다. 파일을 업로드했는지 확인하세요.")
    exit()

# Colab 환경을 위해 프레임 처리 개수를 제한하고 출력 주기를 설정합니다.
frame_count = 0
MAX_FRAMES_TO_PROCESS = 150 # 최대 150 프레임만 처리
DISPLAY_EVERY_N_FRAMES = 20 # 20 프레임마다 결과 출력

print(f"Farneback 광학 흐름 추적 시작 (최대 {MAX_FRAMES_TO_PROCESS} 프레임, {DISPLAY_EVERY_N_FRAMES} 프레임마다 출력)...")


# --- 3. 비디오 처리 루프 ---
while cap.isOpened() and frame_count < MAX_FRAMES_TO_PROCESS:
    ret, frame = cap.read()
    if not ret:
        print("비디오 스트림 종료.")
        break

    # 옵티컬 플로우는 그레이스케일 이미지에서 계산됨
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 최초 프레임의 경우
    if prev is None:
        prev = gray # 첫 프레임을 '이전 프레임'으로 저장
    else:
        # Farneback 옵티컬 플로우 (Dense Optical Flow) 계산
        # Farneback 알고리즘은 모든 픽셀에 대한 움직임 벡터를 계산합니다.
        # flow 변수는 (H, W, 2) 형태의 NumPy 배열이며, 각 픽셀의 (dx, dy) 벡터를 담고 있습니다.
        flow = cv2.calcOpticalFlowFarneback(prev, gray, None,
                                            0.5, # 이미지 피라미드 스케일(각 단계에서 이미지 50% 축소)
                                            3,   # 피라미드 레벨 수(3단계 피라미드 생성)
                                                 # 원본 >> 50% >> 25% (큰 움직임도 잡아냄)
                                            15,  # 윈도우 크기 (평균 이동을 위한 이웃 픽셀 수)
                                                 # 15*15 px 영역에서 계산
                                                 # 크면 부드러워지면서 세밀함 감소
                                                 # 작으면 세밀하지만 노이즈 증가
                                            3,   # 반복 횟수
                                            5,   # 다항식 확장 크기 (픽셀 이웃을 다항식으로 근사)
                                                 # 5*5 영역 (5는 빠른 처리용, 일반적 7)
                                            1.1, # 시그마 값 (가우시안 시그마- 가우시안 블러 >> 노이즈 제거)
                                            cv2.OPTFLOW_FARNEBACK_GAUSSIAN) # 플래그

        # flow.shape = (h, w, 2) >> flow[y, x] = [dx, dy] dx: x방향 이동(px). dy : y방향 이동
        # 계산된 플로 결과 시각화 함수 호출
        drawFlow(frame, flow)

        # 다음 루프를 위해 현재 프레임을 '이전 프레임'으로 이월
        prev = gray

    # Colab 출력: 특정 간격의 프레임만 표시
    if frame_count % DISPLAY_EVERY_N_FRAMES == 0:
        print(f"\n--- Frame {frame_count} Farneback 광학 흐름 결과 ---")
        cv2_imshow(frame) # 추적 결과 프레임 표시
        time.sleep(1) # 출력이 빠르게 지나가는 것을 방지하기 위해 잠시 대기

    # ESC 키 처리는 Colab에서 불가능하므로 프레임 수 제한으로 대체됩니다.
    # if cv2.waitKey(delay) == 27:
    #     break

    frame_count += 1

# --- 4. 종료 및 정리 ---
print("\n비디오 처리 완료 및 정리.")
cap.release()
# 모든 OpenCV 창 닫기 (Colab에서는 필요 없음)
# cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.